# 0.0 Load libraries

In [1]:
from PyPDF2   import PdfFileMerger
from openpyxl import Workbook
from openpyxl import load_workbook

import os
import tabula
import pandas as pd
import re
import pdfplumber
import numpy as np

## 0.1. Auxiliary functions

In [ ]:
# Improve data set visualization, opening all columns and rows. 
# Though this consumes more from the computer, it is important for this project, to check where 
# the informatio is on the data set

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

# 1. Extract data from PDF's  

In this task, there are three main separate documents where the information is stored: 
1. **Race data:** Here the PDF stores the race information (e.g. boat class, weight class, final, date, country, rank,split distance, split velocity, split stroke rate);  
2. **Results:** Here, we can find more information on the race results (e.g. boat class, weight class, final, date, country, rank, lane, name, seat position, 500m split times, and total time every 500m [500m, 1000m, 1500m, and 2000m]);  
3. **Media start list:** Here, we can find information of the crews, before the race happened (e.g. e.g. boat class, weight class, final, date, country, lane, name, date of birth, seat position).

## 1.1. Race data files

In [ ]:
%%time

# naming the directory
dirname = '/media/thiago/THIAGO1TB/Thiago/2021/Academico/GABEF/post_doc/Projeto/data/2022/1x/sr/race_data/'

# providing file extension
ext = ('.PDF')

# A function to collect the heading information from the PDFs
def heading(full):
    venue = full[0].split()[0]
    boat_class = full[1].split()[-1]
    final = full[2].split()[-1]
    race_date = full[3][4:15]

    # Depending on the boat class, the amount of  information is different.
    # Then, it is necessary to establish format of the code that is going to be used.
    boat =[]
    for i in boat_class:
        if i == '8':
            boat=8
        elif i == '4':
            boat=4
        elif i == '2':
            boat=2
        elif i == '1':
            boat=1

    return boat_class, venue, boat, final, race_date

# iterating over all PDF files in the directory, processing data to create variables of interest;
# some PDF files have more than one page. The code handles that as well.
# This code is processing two variables: velocity and stroke rate. This is the reason why you will find the rank being concatenated somewhere in the code(because the code separates the velocity and the stroke rate in different tables).
for pdfname in os.listdir(dirname):
    if pdfname.endswith(ext):
        pdf = dirname + pdfname

        pdf_a = pdfplumber.open(pdf) # Opens file
        page1 = pdf_a.pages[0] # Create page
        text1 = page1.extract_text() # Extract content
        full1 = text1.split('\n') # Split text
        races1 = full1[8:48] # Separate content of interest

        if races1[39][0:4] == '2000': # Determine whether the file has one or more pages.

            aux1 = full1[5].split()[1:]

            # Creating the variable RANK.This part of the code will identify the boat's ranks in the race.
            # Each list present a funcionality. As not all races have six boats, the code need to handle this particularity.
            rank=[] # Race full, with six boats in it;
            rank_aux2 = [np.nan, np.nan, np.nan, np.nan] # When there are two boats in the race, four na's are included;
            rank_aux3 = [np.nan, np.nan, np.nan]# When there are three boats in the race, three na's are included
            rank_aux4 = [np.nan, np.nan] # When there are four boats in the race, two na's are included
            rank_aux5 = [np.nan] # When there are five boats in the race, one na's is included
            rank_a=[] # This list is to be used when there are na's in the rank.

            for j in aux1: #Here, the code checks whether the race is full (6 boats), or is incomplete, and fills
                            # the empty lots with na's
                rank.append(j)

            if len(rank) == 2:
                rank.append(rank_aux2)

                for p in rank:#this for loop removes the na's nested list to form a single list, with 6 items.
                    if type(p) != list:
                        rank_a.append(p)

                    else:
                        for na in p:
                            rank_a.append(na)

                        header_rank=rank_a+rank_a

            elif len(rank) == 3:
                rank.append(rank_aux3)
                # rank_a.append(rank_aux3)

                for p in rank:
                    if type(p) != list:
                        rank_a.append(p)

                    else:
                        for na in p:
                            rank_a.append(na)

                        header_rank=rank_a+rank_a

            elif len(rank) == 4:
                rank.append(rank_aux4)

                for p in rank:
                    if type(p) != list:
                        rank_a.append(p)

                    else:
                        for na in p:
                            rank_a.append(na)

                        header_rank=rank_a+rank_a


            elif len(rank) == 5:
                rank.append(rank_aux5)

                for p in rank:
                    if type(p) != list:
                        rank_a.append(p)

                    else:
                        for na in p:
                            rank_a.append(na)

                        header_rank=rank_a+rank_a

            else:
                header_rank=rank+rank


            # Creating COUNTRY. Similarly to the rank list, here is to create a list with the country names.
            # Here, I have used the same procedures, except that I have organized the COUNTRIES this time.
            aux2 = full1[4].split()[1:]

            # Create empty lists and lists filled with np.nan
            country = []
            country_aux2 = [np.nan, np.nan, np.nan, np.nan]
            country_aux3 = [np.nan, np.nan, np.nan]
            country_aux4 = [np.nan, np.nan]
            country_aux5 = [np.nan]
            country_a  =   []
            country_b  =   []

            #Iterate over the country names to append them to main list
            for j in aux2:
                country.append(j)

            if len(country) == 2:
                country.append  (country_aux2)

                for element in country:#Again, the for loop aims to unnest the list inside the list.
                    if type(element) != list:
                        country_a.append(element)

                    else:
                        for na in element:
                            country_a.append(na)

                country_list = country_a + country_a
            
            elif len(country) == 3:
                country.append  (country_aux3)

                for element in country:
                    if type(element) != list:
                        country_a.append(element)

                    else:
                        for na in element:
                            country_a.append(na)

                country_list = country_a + country_a

            elif len(country) == 4:
                country.append  (country_aux4)

                for element in country:
                    if type(element) != list:
                        country_a.append(element)
                    else:
                        for na in element:
                            country_a.append(na)

                country_list = country_a + country_a

            elif len(country) == 5:
                country.append  (country_aux5)

                for element in country:
                    if type(element) != list:
                        country_a.append(element)
                else:
                    for na in element:
                        country_a.append(na)

                country_list = country_a + country_a

            else:
                country_list = country + country

            # Creating LANE. Here, I have created the LANE variable. Since all races have six lanes, there's no 
            # need for processing.
            lanes=[1,2,3,4,5,6,1,2,3,4,5,6]

            # Separating velocity and stroke rate data. The PDF file provides a table where the data on the
            # VELOCITY and STROKE RATE are parallell, but I needed them to be separate for analysis. Therefore, I
            
            split_dist=[]
            vel_1st = []
            vel_2nd = []
            vel_3rd = []
            vel_4th = []
            vel_5th = []
            vel_6th = []

            spm_1st = []
            spm_2nd = []
            spm_3rd = []
            spm_4th = []
            spm_5th = []
            spm_6th = []
            
            # Considering that not all the races are complete (six boats), I implemented try/except technique
            # to allow the code to run regardless of the race being full or not.
            # Note that velocity is on odd columns whereas the stroke rate is on even columns.
            for i in races1:
                try:
                    split_dist.append(i.split()[0])
                except:
                    pass
                try:
                    vel_1st.append(i.split()[1])
                except:
                    pass
                try:
                    vel_2nd.append(i.split()[3])
                except:
                    pass
                try:
                    vel_3rd.append(i.split()[5])
                except:
                    pass
                try:
                    vel_4th.append(i.split()[7])
                except:
                    pass
                try:
                    vel_5th.append(i.split()[9])
                except:
                    pass
                try:
                    vel_6th.append(i.split()[11])
                except:
                    pass
                
                try:
                    spm_1st.append(i.split()[2])
                except:
                    pass
                try:
                    spm_2nd.append(i.split()[4])
                except:
                    pass
                try:
                    spm_3rd.append(i.split()[6])
                except:
                    pass
                try:
                    spm_4th.append(i.split()[8])
                except:
                    pass
                try:
                    spm_5th.append(i.split()[10])
                except:
                    pass
                try:
                    spm_6th.append(i.split()[12])
                except:
                    pass

            a=pd.Series(split_dist)
            b=pd.Series(vel_1st) 
            c=pd.Series(vel_2nd) 
            d=pd.Series(vel_3rd) 
            e=pd.Series(vel_4th) 
            f=pd.Series(vel_5th) 
            g=pd.Series(vel_6th) 
            h=pd.Series(spm_1st) 
            i=pd.Series(spm_2nd) 
            j=pd.Series(spm_3rd) 
            k=pd.Series(spm_4th) 
            l=pd.Series(spm_5th) 
            m=pd.Series(spm_6th) 

            # Concatenanting all the series to make a data frame.
            ll = pd.concat( [a,b,c,d,e,f,g,h,i,j,k,l,m], axis = 1 )

            df = pd.DataFrame(ll)

            # Start building the final data frame
            df1 = df.T
            df_header = df1.iloc[0]
            df2 = df1.iloc[1:]
            df2.columns=df_header
            df2['rank'] = header_rank
            df2['lanes']= lanes
            df2['country'] = country_list
            df3 = df2.iloc[:6,:]
            df3 = df3.reset_index()
            df4 = df2.iloc[6:,:]
            df4 = df4.reset_index()
            df5 = pd.concat([df3,df4], axis=1)

            df_head = heading(full1)

            df5['age_group'] = df_head[0]
            df5['place'] = df_head[1]
            df5['final'] = df_head[3]
            df5['race_date'] = df_head[4]

            # Preparing the file name to save it locally
            
            csv_name = df_head[0] + '_' + df_head[3] + '_' + df_head[4][-4:] + '.csv'
            csv_path = dirname + csv_name

            # Sending the file to local directory. Note that so far the processing dealt with a single pagePDF.
            df5.to_csv(csv_path, encoding='utf-8', index=False)

        else: #From here, all processing relates to multiple page PDF.
            page2 = pdf_a.pages[1]
            text2 = page2.extract_text()
            full2 = text2.split('\n')
            full2[8]
            races2 = full2[8]
            races3 = full1[8:47]

            q=pd.Series(races3)
            w=pd.Series(races2)
            races_format = pd.concat([q,w])

            aux1 = full1[5].split()[1:]

            #Creating RANK
            rank=[]
            rank_aux2 = [np.nan, np.nan, np.nan, np.nan]
            rank_aux3 = [np.nan, np.nan, np.nan]
            rank_aux4 = [np.nan, np.nan]
            rank_aux5 = [np.nan] 
            rank_a=[]

            for j in aux1:
                rank.append(j)

            if len(rank) == 2:
                rank.append(rank_aux2)

                for p in rank:
                    if type(p) != list:
                        rank_a.append(p)

                    else:
                        for na in p:
                            rank_a.append(na)

                        header_rank=rank_a+rank_a

            elif len(rank) == 3:
                rank.append(rank_aux3)

                for p in rank:
                    if type(p) != list:
                        rank_a.append(p)

                    else:
                        for na in p:
                            rank_a.append(na)

                        header_rank=rank_a+rank_a

            elif len(rank) == 4:
                rank.append(rank_aux4)

                for p in rank:
                    if type(p) != list:
                        rank_a.append(p)

                    else:
                        for na in p:
                            rank_a.append(na)

                        header_rank=rank_a+rank_a


            elif len(rank) == 5:
                rank.append(rank_aux5)
                # rank_a.append(rank_aux5)

                for p in rank:
                    if type(p) != list:
                        rank_a.append(p)

                    else:
                        for na in p:
                            rank_a.append(na)

                        header_rank=rank_a+rank_a

            else:
                header_rank=rank+rank

            # Creating COUNTRY
            aux2 = full1[4].split()[1:]

            # Create empty lists and lists filled with np.nan
            country = []
            country_aux4 = [np.nan, np.nan, np.nan, np.nan]
            country_aux3 = [np.nan, np.nan, np.nan]
            country_aux4 = [np.nan, np.nan]
            country_aux5 = [np.nan]
            country_a  =   []

            #Iterate over the country names to append them to main list
            for j in aux2:
                country.append(j)

            if len(country) == 2:
                country.append  (country_aux2)

                for element in country:
                    if type(element) != list:
                        country_a.append(element)

                    else:
                        for na in element:
                            country_a.append(na)

                country_list = country_a + country_a

            elif len(country) == 3:
                country.append  (country_aux3)

                for element in country:
                    if type(element) != list:
                        country_a.append(element)

                    else:
                        for na in element:
                            country_a.append(na)

                country_list = country_a + country_a

            elif len(country) == 4:
                country.append  (country_aux4)

                for element in country:
                    if type(element) != list:
                        country_a.append(element)
                    else:
                        for na in element:
                            country_a.append(na)

                country_list = country_a + country_a

            elif len(country) == 5:
                country.append  (country_aux5)

                for element in country:
                    if type(element) != list:
                        country_a.append(element)
                else:
                    for na in element:
                        country_a.append(na)

                country_list = country_a + country_a

            else:
                country_list = country + country

            # Creating LANE
            lanes=[1,2,3,4,5,6,1,2,3,4,5,6]

            # Separating velocity and stroke rate data
            split_dist=[]
            vel_1st = []
            vel_2nd = []
            vel_3rd = []
            vel_4th = []
            vel_5th = []
            vel_6th = []

            spm_1st = []
            spm_2nd = []
            spm_3rd = []
            spm_4th = []
            spm_5th = []
            spm_6th = []

            for i in races_format:
                try:
                    split_dist.append(i.split()[0])
                except:
                    pass
                try:
                    vel_1st.append(i.split()[1])
                except:
                    pass
                try:
                    vel_2nd.append(i.split()[3])
                except:
                    pass
                try:
                    vel_3rd.append(i.split()[5])
                except:
                    pass
                try:
                    vel_4th.append(i.split()[7])
                except:
                    pass
                try:
                    vel_5th.append(i.split()[9])
                except:
                    pass
                try:
                    vel_6th.append(i.split()[11])
                except:
                    pass
                
                try:
                    spm_1st.append(i.split()[2])
                except:
                    pass
                try:
                    spm_2nd.append(i.split()[4])
                except:
                    pass
                try:
                    spm_3rd.append(i.split()[6])
                except:
                    pass
                try:
                    spm_4th.append(i.split()[8])
                except:
                    pass
                try:
                    spm_5th.append(i.split()[10])
                except:
                    pass
                try:
                    spm_6th.append(i.split()[12])
                except:
                    pass

            a=pd.Series(split_dist)
            b=pd.Series(vel_1st) 
            c=pd.Series(vel_2nd) 
            d=pd.Series(vel_3rd) 
            e=pd.Series(vel_4th) 
            f=pd.Series(vel_5th) 
            g=pd.Series(vel_6th) 
            h=pd.Series(spm_1st) 
            i=pd.Series(spm_2nd) 
            j=pd.Series(spm_3rd) 
            k=pd.Series(spm_4th) 
            l=pd.Series(spm_5th) 
            m=pd.Series(spm_6th) 

            # Concatenanting all the series to make a data frame.
            ll = pd.concat( [a,b,c,d,e,f,g,h,i,j,k,l,m], axis = 1 )
            
            df = pd.DataFrame(ll)

            df1 = df.T
            df_header = df1.iloc[0]
            df2 = df1.iloc[1:]
            df2.columns=df_header
            df2['rank'] = header_rank
            df2['lanes']= lanes
            df2['country'] = country_list
            df3 = df2.iloc[:6,:]
            df3 = df3.reset_index()
            df4 = df2.iloc[6:,:]
            df4 = df4.reset_index()
            df5 = pd.concat([df3,df4], axis=1)

            df_head = heading(full1)

            df5['age_group'] = df_head[0]
            df5['place'] = df_head[1]
            df5['final'] = df_head[3]
            df5['race_date'] = df_head[4]


            # Preparing the file name to save it locally
            
            csv_name = df_head[0] + '_' + df_head[3] + '_' + df_head[4][-4:] + '.csv'
            csv_path = dirname + csv_name

            # Sending the file to local directory. Note that so far the processing dealt with a single pagePDF.
            df5.to_csv(csv_path, encoding='utf-8', index=False)

## 1.2. Results files

### General  information:  

Here, I have scraped the data from the PDFs, to form a single data frame to be used in the analysis.

### Iterate over PDF files in directory and generate .csv files for processing.

* Here, I have scraped the data from the PDFs, to form a single data frame to be used in the analysis.  

* Retrieve data from PDF and save them in .csv for processing.

In [ ]:
%%time

# Defining heading data
def heading(full):
        venue = full[0].split()[0]
        boat_class = full[1].split()[-1]
        final = full[2].split()[-1]
        race_date = full[3][4:15]

        # Establish format behaviour
        boat =[]
        for i in boat_class:
                if i == '8':
                        boat=8
                elif i == '4':
                        boat=4
                elif i == '2':
                        boat=2
                elif i == '1':
                        boat=1

        return boat_class, venue, boat, final, race_date


# Here I have functions to isolate the data for each rank position, for each boat class.  
def first_place_single(races):
        try:
                rank = races   [0].split()[0]
                lane = races   [0].split()[1]
                country = races[0].split()[2]
                
                surname_b = races[0].split()[3]
                name_b = races[0].split()[4]

                t500=races [0].split()[-8]
                t1000=races[0].split()[-6]
                t1500=races[0].split()[-4]
                t2000=races[0].split()[-2]

                f_name_b = surname_b + ' ' + name_b

                return [rank, lane, country,  
                        f_name_b, 
                        t500,t1000,t1500,t2000]
        except:
                pass

def second_place_single(races):
        try:
                rank =    races[3].split()[0]
                lane =    races[3].split()[1]
                country = races[3].split()[2]

                surname_b = races[3].split()[3]
                name_b = races[3].split()[4]

                t500=races [3].split() [-8]
                t1000=races[3].split()[-6]
                t1500=races[3].split()[-4]
                t2000=races[3].split()[-2]

                f_name_b = surname_b + ' ' + name_b

                return [rank, lane, country,  
                        f_name_b,
                        t500,t1000,t1500,t2000]

        except:
                pass

def third_place_single(races):
        try:
                rank = races   [6].split()[0]
                lane = races   [6].split()[1]
                country = races[6].split()[2]

                surname_b = races[6].split()[3]
                name_b = races[6].split()[4]

                t500=races [6].split()[-8]
                t1000=races[6].split()[-6]
                t1500=races[6].split()[-4]
                t2000=races[6].split()[-2]

                f_name_b = surname_b + ' ' + name_b

                return [rank, lane, country, 
                        f_name_b, 
                        t500,t1000,t1500,t2000]

        except:
                pass

def fourth_place_single(races):
        try:
                rank = races   [9].split()[0]
                lane = races   [9].split()[1]
                country = races[9].split()[2]

                surname_b = races[9].split()[3]
                name_b = races[9].split()[4]

                t500=races [9].split()[-8]
                t1000=races[9].split()[-6]
                t1500=races[9].split()[-4]
                t2000=races[9].split()[-2]

                f_name_b = surname_b + ' ' + name_b

                return [rank, lane, country,  
                        f_name_b,
                        t500,t1000,t1500,t2000]

        except:
                pass

def fifth_place_single(races):
        try:
                rank = races   [12].split()[0]
                lane = races   [12].split()[1]
                country = races[12].split()[2]

                surname_b = races[12].split()[3]
                name_b = races[12].split()[4]

                t500=races [12].split()[-8]
                t1000=races[12].split()[-6]
                t1500=races[12].split()[-4]
                t2000=races[12].split()[-2]

                f_name_b = surname_b + ' ' + name_b

                return [rank, lane, country,  
                        f_name_b,
                        t500,t1000,t1500,t2000]

        except:
                pass

def sixth_place_single(races):
        try:
                rank = races   [15].split()[0]
                lane = races   [15].split()[1]
                country = races[15].split()[2]

                surname_b = races[15].split()[3]
                name_b = races[15].split()[4]

                t500=races [15].split()[-8]
                t1000=races[15].split()[-6]
                t1500=races[15].split()[-4]
                t2000=races[15].split()[-2]

                f_name_b = surname_b + ' ' + name_b

                return [rank, lane, country,  
                        f_name_b,
                        t500,t1000,t1500,t2000]

        except:
                pass


def first_place_double(races):
        try:
                rank = races   [0].split()[0]
                lane = races   [0].split()[1]
                country = races[0].split()[2]
                seat_b = races [0].split()[3]
                seat_s = races [1].split()[0]

                surname_b = races[0].split()[4]
                surname_s = races[1].split()[1]

                name_b = races[0].split()[5]
                name_s = races[1].split()[2]

                t500=races [0].split()[-8]
                t1000=races[0].split()[-6]
                t1500=races[0].split()[-4]
                t2000=races[0].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_s = surname_s + ' ' + name_s

                return [rank, lane, country, seat_b,seat_s, 
                        f_name_b,f_name_s, 
                        t500,t1000,t1500,t2000]
        except:
                pass

def second_place_double(races):
        try:
                rank = races[2].split()[0]
                lane = races[2].split()[1]
                country = races[2].split()[2]
                seat_b = races[2].split()[3]
                seat_s = races[3].split()[0]

                surname_b = races[2].split()[4]
                surname_s = races[3].split()[1]

                name_b = races[2].split()[5]
                name_s = races[3].split()[2]

                t500=races[2].split() [-8]
                t1000=races[2].split()[-6]
                t1500=races[2].split()[-4]
                t2000=races[2].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_s = surname_s + ' ' + name_s

                return [rank, lane, country, seat_b,seat_s, 
                        f_name_b,f_name_s, 
                        t500,t1000,t1500,t2000]

        except:
                pass

def third_place_double(races):
        try:
                rank = races[5].split()[0]
                lane = races[5].split()[1]
                country = races[5].split()[2]
                seat_b = races[5].split()[3]
                seat_s = races[6].split()[0]

                surname_b = races[5].split()[4]
                surname_s = races[6].split()[1]

                name_b = races[5].split()[5]
                name_s = races[6].split()[2]

                t500=races[5].split()[-8]
                t1000=races[5].split()[-6]
                t1500=races[5].split()[-4]
                t2000=races[5].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_s = surname_s + ' ' + name_s

                return [rank, lane, country, seat_b,seat_s, 
                        f_name_b,f_name_s, 
                        t500,t1000,t1500,t2000]

        except:
                pass

def fourth_place_double(races):
        try:
                rank = races[8].split()[0]
                lane = races[8].split()[1]
                country = races[8].split()[2]
                seat_b = races[8].split()[3]
                seat_s = races[9].split()[0]

                surname_b = races[8].split()[4]
                surname_s = races[9].split()[1]

                name_b = races[8].split()[5]
                name_s = races[9].split()[2]

                t500=races[8].split()[-8]
                t1000=races[8].split()[-6]
                t1500=races[8].split()[-4]
                t2000=races[8].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_s = surname_s + ' ' + name_s

                return [rank, lane, country, seat_b,seat_s, 
                        f_name_b,f_name_s, 
                        t500,t1000,t1500,t2000]

        except:
                pass

def fifth_place_double(races):
        try:
                rank = races[11].split()[0]
                lane = races[11].split()[1]
                country = races[11].split()[2]
                seat_b = races[11].split()[3]
                seat_s = races[12].split()[0]

                surname_b = races[11].split()[4]
                surname_s = races[12].split()[1]

                name_b = races[11].split()[5]
                name_s = races[12].split()[2]

                t500=races[11].split()[-8]
                t1000=races[11].split()[-6]
                t1500=races[11].split()[-4]
                t2000=races[11].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_s = surname_s + ' ' + name_s

                return [rank, lane, country, seat_b,seat_s, 
                        f_name_b,f_name_s, 
                        t500,t1000,t1500,t2000]

        except:
                pass

def sixth_place_double(races):
        try:
                rank = races[14].split()[0]
                lane = races[14].split()[1]
                country = races[14].split()[2]
                seat_b = races[14].split()[3]
                seat_s = races[15].split()[0]

                surname_b = races[14].split()[4]
                surname_s = races[15].split()[1]

                name_b = races[14].split()[5]
                name_s = races[15].split()[2]

                t500=races[14].split()[-8]
                t1000=races[14].split()[-6]
                t1500=races[14].split()[-4]
                t2000=races[14].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_s = surname_s + ' ' + name_s

                return [rank, lane, country, seat_b,seat_s, 
                        f_name_b,f_name_s, 
                        t500,t1000,t1500,t2000]

        except:
                pass


def first_place_four(races):
        try:
                rank = races[0].split()[0]
                lane = races[0].split()[1]
                country = races[0].split()[2]

                seat_b = races[0].split()[3]
                seat_2 = races[1].split()[0]
                seat_3 = races[2].split()[0]
                seat_s = races[3].split()[0]

                surname_b = races[0].split()[4]
                surname_2 = races[1].split()[1]
                surname_3 = races[2].split()[1]
                surname_s = races[3].split()[1]

                name_b = races[0].split()[5]
                name_2 = races[1].split()[2]
                name_3 = races[2].split()[2]
                name_s = races[3].split()[2]
        
                t500=races[0].split()[-8]
                t1000=races[0].split()[-6]
                t1500=races[0].split()[-4]
                t2000=races[0].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_s = surname_s + ' ' + name_s

                return [rank, lane, country, seat_b,seat_2, seat_3,seat_s, 
                        f_name_b, f_name_2, f_name_3, f_name_s, 
                        t500,t1000,t1500,t2000]

        except:
                pass

def second_place_four(races):
        try:
                rank = races[4].split()[0]
                lane = races[4].split()[1]
                country = races[4].split()[2]

                seat_b = races[4].split()[3]
                seat_2 = races[5].split()[0]
                seat_3 = races[6].split()[0]
                seat_s = races[7].split()[0]

                surname_b = races[4].split()[4]
                surname_2 = races[5].split()[1]
                surname_3 = races[6].split()[1]
                surname_s = races[7].split()[1]

                name_b = races[4].split()[5]
                name_2 = races[5].split()[2]
                name_3 = races[6].split()[2]
                name_s = races[7].split()[2]

                t500=races[4].split()[-8]
                t1000=races[4].split()[-6]
                t1500=races[4].split()[-4]
                t2000=races[4].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_s = surname_s + ' ' + name_s
        
                return [rank, lane, country, seat_b,seat_2, seat_3,seat_s, 
                        f_name_b, f_name_2, f_name_3, f_name_s, 
                        t500,t1000,t1500,t2000]

        except:
                pass

def third_place_four(races):
        try:
                rank = races[8].split()[0]
                lane = races[8].split()[1]
                country = races[8].split()[2]

                seat_b = races[8].split()[3]
                seat_2 = races[9].split()[0]
                seat_3 = races[10].split()[0]
                seat_s = races[11].split()[0]

                surname_b = races[8].split()[4]
                surname_2 = races[9].split()[1]
                surname_3 = races[10].split()[1]
                surname_s = races[11].split()[1]

                name_b = races[8].split()[5]
                name_2 = races[9].split()[2]
                name_3 = races[10].split()[2]
                name_s = races[11].split()[2]

                t500=races[8].split()[-8]
                t1000=races[8].split()[-6]
                t1500=races[8].split()[-4]
                t2000=races[8].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_s = surname_s + ' ' + name_s

                return [rank, lane, country, seat_b,seat_2, seat_3,seat_s, 
                        f_name_b, f_name_2, f_name_3, f_name_s, 
                        t500,t1000,t1500,t2000]

        except:
                pass

def fourth_place_four(races):
        try:
                rank = races[12].split()[0]
                lane = races[12].split()[1]
                country = races[12].split()[2]

                seat_b = races[12].split()[3]
                seat_2 = races[13].split()[0]
                seat_3 = races[14].split()[0]
                seat_s = races[15].split()[0]

                surname_b = races[12].split()[4]
                surname_2 = races[13].split()[1]
                surname_3 = races[14].split()[1]
                surname_s = races[15].split()[1]

                name_b = races[12].split()[5]
                name_2 = races[13].split()[2]
                name_3 = races[14].split()[2]
                name_s = races[15].split()[2]

                t500=races[12].split()[-8]
                t1000=races[12].split()[-6]
                t1500=races[12].split()[-4]
                t2000=races[12].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_s = surname_s + ' ' + name_s

                return [rank, lane, country, seat_b,seat_2, seat_3,seat_s, 
                        f_name_b, f_name_2, f_name_3, f_name_s, 
                        t500,t1000,t1500,t2000]

        except:
                pass

def fifth_place_four(races):
        try:
                rank = races[16].split()[0]
                lane = races[16].split()[1]
                country = races[16].split()[2]

                seat_b = races[16].split()[3]
                seat_2 = races[17].split()[0]
                seat_3 = races[18].split()[0]
                seat_s = races[19].split()[0]

                surname_b = races[16].split()[4]
                surname_2 = races[17].split()[1]
                surname_3 = races[18].split()[1]
                surname_s = races[19].split()[1]

                name_b = races[16].split()[5]
                name_2 = races[17].split()[2]
                name_3 = races[18].split()[2]
                name_s = races[19].split()[2]

                t500=races[16].split()[-8]
                t1000=races[16].split()[-6]
                t1500=races[16].split()[-4]
                t2000=races[16].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_s = surname_s + ' ' + name_s

                return [rank, lane, country, seat_b,seat_2, seat_3,seat_s, 
                        f_name_b, f_name_2, f_name_3, f_name_s, 
                        t500,t1000,t1500,t2000]

        except:
                pass

def sixth_place_four(races):
        try:
                rank = races[20].split()[0]
                lane = races[20].split()[1]
                country = races[20].split()[2]

                seat_b = races[20].split()[3]
                seat_2 = races[21].split()[0]
                seat_3 = races[22].split()[0]
                seat_s = races[23].split()[0]

                surname_b = races[20].split()[4]
                surname_2 = races[21].split()[1]
                surname_3 = races[22].split()[1]
                surname_s = races[23].split()[1]

                name_b = races[20].split()[5]
                name_2 = races[21].split()[2]
                name_3 = races[22].split()[2]
                name_s = races[23].split()[2]

                t500=races[20].split()[-8]
                t1000=races[20].split()[-6]
                t1500=races[20].split()[-4]
                t2000=races[20].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_s = surname_s + ' ' + name_s

                return [rank, lane, country, seat_b,seat_2, seat_3,seat_s, 
                        f_name_b, f_name_2, f_name_3, f_name_s, 
                        t500,t1000,t1500,t2000]

        except:
                pass


def first_place_eight(races):
        try:
                rank = races[0].split()[0]
                lane = races[0].split()[1]
                country = races[0].split()[2]

                seat_b = races[0].split()[3]
                seat_2 = races[1].split()[0]
                seat_3 = races[2].split()[0]
                seat_4 = races[3].split()[0]
                seat_5 = races[4].split()[0]
                seat_6 = races[5].split()[0]
                seat_7 = races[6].split()[0]
                seat_s = races[7].split()[0]
                seat_c = races[8].split()[0]

                surname_b = races[0].split()[4]
                surname_2 = races[1].split()[1]
                surname_3 = races[2].split()[1]
                surname_4 = races[3].split()[1]
                surname_5 = races[4].split()[1]
                surname_6 = races[5].split()[1]
                surname_7 = races[6].split()[1]
                surname_s = races[7].split()[1]
                surname_c = races[8].split()[1]

                name_b = races[0].split()[5]
                name_2 = races[1].split()[2]
                name_3 = races[2].split()[2]
                name_4 = races[3].split()[2]
                name_5 = races[4].split()[2]
                name_6 = races[5].split()[2]
                name_7 = races[6].split()[2]
                name_s = races[7].split()[2]
                name_c = races[8].split()[2]

                t500=races[0].split()[-8]
                t1000=races[0].split()[-6]
                t1500=races[0].split()[-4]
                t2000=races[0].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_4 = surname_4 + ' ' + name_4
                f_name_5 = surname_5 + ' ' + name_5
                f_name_6 = surname_6 + ' ' + name_6
                f_name_7 = surname_7 + ' ' + name_7
                f_name_s = surname_s + ' ' + name_s
                f_name_c = surname_c + ' ' + name_c

                return [rank, lane, country, seat_b,seat_2,seat_3,
                        seat_4,seat_5,seat_6,seat_7,seat_s, seat_c, 
                        f_name_b,f_name_2,f_name_3,f_name_4,f_name_5,
                        f_name_6,f_name_7,f_name_s,f_name_c, 
                        t500,t1000,t1500,t2000]
        except:
                pass

def second_place_eight(races):
        try:
                rank = races[9].split()[0]
                lane = races[9].split()[1]
                country = races[9].split()[2]

                seat_b = races[9].split()[3]
                seat_2 = races[10].split()[0]
                seat_3 = races[11].split()[0]
                seat_4 = races[12].split()[0]
                seat_5 = races[13].split()[0]
                seat_6 = races[14].split()[0]
                seat_7 = races[15].split()[0]
                seat_s = races[16].split()[0]
                seat_c = races[17].split()[0]

                surname_b = races[9].split()[4]
                surname_2 = races[10].split()[1]
                surname_3 = races[11].split()[1]
                surname_4 = races[12].split()[1]
                surname_5 = races[13].split()[1]
                surname_6 = races[14].split()[1]
                surname_7 = races[15].split()[1]
                surname_s = races[16].split()[1]
                surname_c = races[17].split()[1]

                name_b = races[9].split()[5]
                name_2 = races[10].split()[2]
                name_3 = races[11].split()[2]
                name_4 = races[12].split()[2]
                name_5 = races[13].split()[2]
                name_6 = races[14].split()[2]
                name_7 = races[15].split()[2]
                name_s = races[16].split()[2]
                name_c = races[17].split()[2]

                t500=races[9].split()[-8]
                t1000=races[9].split()[-6]
                t1500=races[9].split()[-4]
                t2000=races[9].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_4 = surname_4 + ' ' + name_4
                f_name_5 = surname_5 + ' ' + name_5
                f_name_6 = surname_6 + ' ' + name_6
                f_name_7 = surname_7 + ' ' + name_7
                f_name_s = surname_s + ' ' + name_s
                f_name_c = surname_c + ' ' + name_c

                return [rank, lane, country, seat_b,seat_2,seat_3,
                        seat_4,seat_5,seat_6,seat_7,seat_s, seat_c, 
                        f_name_b,f_name_2,f_name_3,f_name_4,f_name_5,
                        f_name_6,f_name_7,f_name_s,f_name_c, 
                        t500,t1000,t1500,t2000]
        except:
                pass

def third_place_eight(races):
        try:
                rank = races[18].split()[0]
                lane = races[18].split()[1]
                country = races[18].split()[2]
                seat_b = races[18].split()[3]

                seat_2 = races[19].split()[0]
                seat_3 = races[20].split()[0]
                seat_4 = races[21].split()[0]
                seat_5 = races[22].split()[0]
                seat_6 = races[23].split()[0]
                seat_7 = races[24].split()[0]
                seat_s = races[25].split()[0]
                seat_c = races[26].split()[0]

                surname_b = races[18].split()[4]
                surname_2 = races[19].split()[1]
                surname_3 = races[20].split()[1]
                surname_4 = races[21].split()[1]
                surname_5 = races[22].split()[1]
                surname_6 = races[23].split()[1]
                surname_7 = races[24].split()[1]
                surname_s = races[25].split()[1]
                surname_c = races[26].split()[1]

                name_b = races[18].split()[5]
                name_2 = races[19].split()[2]
                name_3 = races[20].split()[2]
                name_4 = races[21].split()[2]
                name_5 = races[22].split()[2]
                name_6 = races[23].split()[2]
                name_7 = races[24].split()[2]
                name_s = races[25].split()[2]
                name_c = races[26].split()[2]

                t500=races[18].split()[-8]
                t1000=races[18].split()[-6]
                t1500=races[18].split()[-4]
                t2000=races[18].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_4 = surname_4 + ' ' + name_4
                f_name_5 = surname_5 + ' ' + name_5
                f_name_6 = surname_6 + ' ' + name_6
                f_name_7 = surname_7 + ' ' + name_7
                f_name_s = surname_s + ' ' + name_s
                f_name_c = surname_c + ' ' + name_c

                return [rank, lane, country, seat_b,seat_2,seat_3,
                        seat_4,seat_5,seat_6,seat_7,seat_s, seat_c, 
                        f_name_b,f_name_2,f_name_3,f_name_4,f_name_5,
                        f_name_6,f_name_7,f_name_s,f_name_c, 
                        t500,t1000,t1500,t2000]
        except:
                pass

def fourth_place_eight(races):
        try:
                rank = races[27].split()[0]
                lane = races[27].split()[1]
                country = races[27].split()[2]

                seat_b = races[27].split()[3]
                seat_2 = races[28].split()[0]
                seat_3 = races[29].split()[0]
                seat_4 = races[30].split()[0]
                seat_5 = races[31].split()[0]
                seat_6 = races[32].split()[0]
                seat_7 = races[33].split()[0]
                seat_s = races[34].split()[0]
                seat_c = races[35].split()[0]

                surname_b = races[27].split()[4]
                surname_2 = races[28].split()[1]
                surname_3 = races[29].split()[1]
                surname_4 = races[30].split()[1]
                surname_5 = races[31].split()[1]
                surname_6 = races[32].split()[1]
                surname_7 = races[33].split()[1]
                surname_s = races[34].split()[1]
                surname_c = races[35].split()[1]

                name_b = races[27].split()[5]
                name_2 = races[28].split()[2]
                name_3 = races[29].split()[2]
                name_4 = races[30].split()[2]
                name_5 = races[31].split()[2]
                name_6 = races[32].split()[2]
                name_7 = races[33].split()[2]
                name_s = races[34].split()[2]
                name_c = races[35].split()[2]

                t500=races[27].split()[-8]
                t1000=races[27].split()[-6]
                t1500=races[27].split()[-4]
                t2000=races[27].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_4 = surname_4 + ' ' + name_4
                f_name_5 = surname_5 + ' ' + name_5
                f_name_6 = surname_6 + ' ' + name_6
                f_name_7 = surname_7 + ' ' + name_7
                f_name_s = surname_s + ' ' + name_s
                f_name_c = surname_c + ' ' + name_c

                return [rank, lane, country, seat_b,seat_2,seat_3,
                        seat_4,seat_5,seat_6,seat_7,seat_s, seat_c, 
                        f_name_b,f_name_2,f_name_3,f_name_4,f_name_5,
                        f_name_6,f_name_7,f_name_s,f_name_c, 
                        t500,t1000,t1500,t2000]
        except:
                pass

def fifth_place_eight(races):
        try:
                rank = races[36].split()[0]
                lane = races[36].split()[1]
                country = races[36].split()[2]

                seat_b = races[36].split()[3]
                seat_2 = races[37].split()[0]
                seat_3 = races[38].split()[0]
                seat_4 = races[39].split()[0]
                seat_5 = races[40].split()[0]
                seat_6 = races[41].split()[0]
                seat_7 = races[42].split()[0]
                seat_s = races[43].split()[0]
                seat_c = races[44].split()[0]

                surname_b = races[36].split()[4]
                surname_2 = races[37].split()[1]
                surname_3 = races[38].split()[1]
                surname_4 = races[39].split()[1]
                surname_5 = races[40].split()[1]
                surname_6 = races[41].split()[1]
                surname_7 = races[42].split()[1]
                surname_s = races[43].split()[1]
                surname_c = races[44].split()[1]

                name_b = races[36].split()[5]
                name_2 = races[37].split()[2]
                name_3 = races[38].split()[2]
                name_4 = races[39].split()[2]
                name_5 = races[40].split()[2]
                name_6 = races[41].split()[2]
                name_7 = races[42].split()[2]
                name_s = races[43].split()[2]
                name_c = races[44].split()[2]

                t500=races[36].split()[-8]
                t1000=races[36].split()[-6]
                t1500=races[36].split()[-4]
                t2000=races[36].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_4 = surname_4 + ' ' + name_4
                f_name_5 = surname_5 + ' ' + name_5
                f_name_6 = surname_6 + ' ' + name_6
                f_name_7 = surname_7 + ' ' + name_7
                f_name_s = surname_s + ' ' + name_s
                f_name_c = surname_c + ' ' + name_c

                return [rank, lane, country, seat_b,seat_2,seat_3,
                        seat_4,seat_5,seat_6,seat_7,seat_s, seat_c, 
                        f_name_b,f_name_2,f_name_3,f_name_4,f_name_5,
                        f_name_6,f_name_7,f_name_s,f_name_c, 
                        t500,t1000,t1500,t2000]
        except:
                pass

def sixth_place_eight(races):
        try:
                rank = races[45].split()[0]
                lane = races[45].split()[1]
                country = races[45].split()[2]

                seat_b = races[45].split()[3]
                seat_2 = races[46].split()[0]
                seat_3 = races[47].split()[0]
                seat_4 = races[48].split()[0]
                seat_5 = races[49].split()[0]
                seat_6 = races[50].split()[0]
                seat_7 = races[51].split()[0]
                seat_s = races[52].split()[0]
                seat_c = races[53].split()[0]

                surname_b = races[45].split()[4]
                surname_2 = races[46].split()[1]
                surname_3 = races[47].split()[1]
                surname_4 = races[48].split()[1]
                surname_5 = races[49].split()[1]
                surname_6 = races[50].split()[1]
                surname_7 = races[51].split()[1]
                surname_s = races[52].split()[1]
                surname_c = races[53].split()[1]

                name_b = races[45].split()[5]
                name_2 = races[46].split()[2]
                name_3 = races[47].split()[2]
                name_4 = races[48].split()[2]
                name_5 = races[49].split()[2]
                name_6 = races[50].split()[2]
                name_7 = races[51].split()[2]
                name_s = races[52].split()[2]
                name_c = races[53].split()[2]

                t500=races[45].split()[-8]
                t1000=races[45].split()[-6]
                t1500=races[45].split()[-4]
                t2000=races[45].split()[-2]

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_4 = surname_4 + ' ' + name_4
                f_name_5 = surname_5 + ' ' + name_5
                f_name_6 = surname_6 + ' ' + name_6
                f_name_7 = surname_7 + ' ' + name_7
                f_name_s = surname_s + ' ' + name_s
                f_name_c = surname_c + ' ' + name_c

                return [rank, lane, country, seat_b,seat_2,seat_3,
                        seat_4,seat_5,seat_6,seat_7,seat_s, seat_c, 
                        f_name_b,f_name_2,f_name_3,f_name_4,f_name_5,
                        f_name_6,f_name_7,f_name_s,f_name_c, 
                        t500,t1000,t1500,t2000]
        except:
                pass


#######################################
####PDF RETRIEVING DATA STARTS HERE####
#######################################

# giving directory name
dirna m e = '/media/thiago/THIAGO1TB/Thiago/2021/Academico/GABEF/post_doc/Projeto/data/2022/1x/u23/results/'

# giving file extension
ext = ('.PDF')

# iterating over all files
for pdfname in os.listdir(dirname):
        if pdfname.endswith(ext):
                pdf = dirname + pdfname

                pdf_a = pdfplumber.open(pdf)
                page = pdf_a.pages[0]
                text = page.extract_text()
                full = text.split('\n')

                df_head = heading(full)

                if df_head[2] == 1:
                        print(pdf)
                        races = full[10:]
                        a = first_place_single( races )
                        b =  s econd_plac e_single( races )
                        c = third_place_single( races )
                        d = fourth_place_single( races )
                        e = fifth_place_single( races )
                        f = sixth_place_single( races )

                        hd = pd.Series( df_head )
                        aa = pd.Series( a )
                        bb = pd.Series( b )
                        cc = pd.Series( c )
                        dd = pd.Series( d )
                        ee = pd.Series( e )
                        ff = pd.Series( f )
                        
                        k = pd.concat( [aa, bb, cc, dd, ee, ff], axis = 1 )
                        l = pd.concat( [hd, k], axis = 1  ) .T

                        csv_name = df_head[0] + '_' + df_head[3] + '_' + df_head[4][-4:] + '.csv'
                        csv_path = dirname + csv_name

                        l.to_csv(csv_path, encoding='utf-8', index=False)
                
                elif df_head[2] == 2:
                        print(pdf)
                        races = full[10:]
                        a = first_place_double(races)
                        b = second_place_double(races)
                        c = third_place_double(races)
                        d = fourth_place_double(races)
                        e = fifth_place_double(races)
                        f = sixth_place_double(races)

                        hd = pd.Series(df_head)
                        aa = pd.Series(a)
                        bb = pd.Series(b)
                        cc = pd.Series(c)
                        dd = pd.Series(d)
                        ee = pd.Series(e)
                        ff = pd.Series(f)
                        
                        k = pd.concat([aa, bb, cc, dd, ee, ff], axis = 1)
                        l = pd.concat([hd, k], axis = 1).T

                        csv_name = df_head[0] + '_' + df_head[3] + '_' + df_head[4][-4:] + '.csv'
                        csv_path = dirname + csv_name

                        l.to_csv(csv_path, encoding='utf-8', index=False)

                elif df_head[2] == 4:
                        print(pdf)
                        races = full[10:]
                        a = first_place_four(races)
                        b = second_place_four(races)
                        c = third_place_four(races)
                        d = fourth_place_four(races)
                        e = fifth_place_four(races)
                        f = sixth_place_four(races)

                        hd = pd.Series(df_head)
                        aa = pd.Series(a)
                        bb = pd.Series(b)
                        cc = pd.Series(c)
                        dd = pd.Series(d) 
                        ee = pd.Series(e)
                        ff = pd.Series(f)

                        k = pd.concat([aa, bb, cc, dd, ee, ff], axis = 1)
                        l = pd.concat([hd, k], axis = 1).T

                        csv_name = df_head[0] + '_' + df_head[3] + '_' + df_head[4][-4:] + '.csv'
                        csv_path = dirname + csv_name

                        l.to_csv(csv_path, encoding='utf-8', index=False)

                elif df_head[2] == 8:
                        print(pdf)
                        races = full[10:]
                        a = first_place_eight(races)
                        b = second_place_eight(races)
                        c = third_place_eight(races)
                        d = fourth_place_eight(races)
                        e = fifth_place_eight(races)
                        f = sixth_place_eight(races)

                        hd = pd.Series(df_head)
                        aa = pd.Series(a)
                        bb = pd.Series(b)
                        cc = pd.Series(c)
                        dd = pd.Series(d)
                        ee = pd.Series(e)
                        ff = pd.Series(f)

                        k = pd.concat([aa, bb, cc, dd, ee, ff], axis = 1)
                        l = pd.concat([hd, k], axis = 1).T

                        csv_name = df_head[0] + '_' + df_head[3] + '_' + df_head[4][-4:] + '.csv'
                        csv_path = dirname + csv_name

                        l.to_csv(csv_path, encoding='utf-8', index=False)

### Formatting the .csv files for processing.

* Here, I had to format the .csv files to be able to merge them into a single file for further analysis.  

In [ ]:
# Directory where the files are located
dirname = '/media/thiago/THIAGO1TB/Thiago/2021/Academico/GABEF/post_doc/Projeto/data/2022/1x/u23/results/'

# What file extension to look for.
ext = ('.csv')

list_aux = [] # Auxiliary list to place the newly formatted data frame.

# iterating over all files
for csvname in os.listdir(dirname):
    if csvname.endswith(ext):
        csv_file = dirname + csvname

        df=pd.read_csv(csv_file)

        # As the files may contain data from single, double, four, or eight boat_class,
        # the conditional if/elif/else handles that
        if df.iloc[0,2] == '1':
            #Creating columns using headings information
            df['age_group'] = df.iloc[0,0]
            df['place'] = df.iloc[0,1]
            df['final'] = df.iloc[0,3]
            df['race_date'] = df.iloc[0,4]
            df = df.drop(0)

            df = df.rename(columns={df.columns[0]: 'rank', df.columns[1]: 'lane', df.columns[2]: 'country',
                                    df.columns[3]: 'name', df.columns[4]: '500m', df.columns[5]: '1000m',
                                    df.columns[6]: '1500m', df.columns[7]: '2000m'})

        elif df.iloc[0,2] == '2':
            # Creating columns using headings information
            df['age_group'] = df.iloc[0,0]
            df['place'] = df.iloc[0,1]
            df['final'] = df.iloc[0,3]
            df['race_date'] = df.iloc[0,4]
            df = df.drop(0)

            # Create a new column, concatenating boat positioning and names df
            df['ath_pos'] = df.apply(lambda x: [str(x[i+3]) + '~' + str(x[i+5]) for i in range (2)] ,axis=1)

            # From the newly created column, separate the athletes and place their names in separate 
            #rows (creating new rows)
            df = df.explode('ath_pos')

            # Separate crewpos from name in two columns
            df[['pos', 'name']] = df['ath_pos'].str.split('~', expand=True)

            # Drop unused columns
            df = df.drop(df.columns[[3,4,5,6]], axis=1)
            df = df.drop(['ath_pos'], axis=1)

            # Rename the dataframe
            df = df.rename(columns={df.columns[0]: 'rank', df.columns[1]: 'lane', df.columns[2]: 'country',
                                    df.columns[3]: '500m', df.columns[4]: '1000m', df.columns[5]: '1500m', 
                                    df.columns[6]: '2000m', 'pos': 'crewpos'})

        elif df.iloc[0,2] == '4':
            # Creating columns using headings information
            df['age_group'] = df.iloc[0,0]
            df['place'] = df.iloc[0,1]
            df['final'] = df.iloc[0,3]
            df['race_date'] = df.iloc[0,4]
            df=df.drop(0)

            # Create a new column, concatenating boat positioning and names
            df['ath_pos'] = df.apply(lambda x: [str(x[i+3]) + '~' + str(x[i+7]) for i in range (4)] ,axis=1)

            # From the newly created column, separate the athletes and place their names in separate rows
            #(creating new rows)
            df = df.explode('ath_pos')

            # Separate crewpos from name in two columns
            df[['pos', 'name']] = df['ath_pos'].str.split('~', expand=True)

            # Drop old, unused columns
            df = df.drop(df.columns[[3,4,5,6,7,8,9,10]], axis=1)
            df = df.drop(['ath_pos'], axis=1)
            # 
            # Rename the dataframe
            df = df.rename(columns={df.columns[0]: 'rank', df.columns[1]: 'lane', df.columns[2]: 'country',
                                    df.columns[3]: '500m', df.columns[4]: '1000m', df.columns[5]: '1500m', 
                                    df.columns[6]: '2000m', 'pos': 'crewpos'})

                    
        elif df.iloc[0,2] == '8':
                    # Creating columns using headings information
            df['age_group'] = df.iloc[0,0]
            df['place'] = df.iloc[0,1]
            df['final'] = df.iloc[0,3]
            df['race_date'] = df.iloc[0,4]
            df=df.drop(0)
            
            # Create a new column, concatenating boat positioning and names
            df['ath_pos'] = df.apply(lambda x: [str(x[i+3]) + '~' + str(x[i+12]) for i in range (9)] ,axis=1)
            
            # From the newly created column, separate the athletes and place their names in separate rows
            # (creating new rows)
            df = df.explode('ath_pos')
            
            # Separate crewpos from name in two columns
            df[['pos', 'name']] = df['ath_pos'].str.split('~', expand=True)
            
            # Drop old, unused columns
            df = df.drop(df.iloc[:, 3:21], axis=1)
            df = df.drop(['ath_pos'], axis=1)
            # 
            # Rename the dataframe
            df = df.rename(columns={df.columns[0]: 'rank', df.columns[1]: 'lane', df.columns[2]: 'country',
                                    df.columns[3]: '500m', df.columns[4]: '1000m', df.columns[5]: '1500m', 
                                    df.columns[6]: '2000m', 'pos': 'crewpos'})
    
        # If the code above fails, print the warning message below:
        else:
            print('Code failed: None of the options')

        #Populate the empty list with the new data frame
        list_aux.append(df)

#Concatenate the dataframes into a new one
df_aux = pd.concat(list_aux)

#Create a varible "code" to be used to merge files
df_aux['code'] = df_aux.apply(lambda x: [str(x[i]) + str(x[i+1]) + str(x[i+2]) + str(x[i+3]) + str(x[i+4]) +
                                str(x[i+5])  for i in range (1)] ,axis=1)
 
# Reordering columns for single boats data
df_aux = df_aux.loc[:,['age_group','place', 'final', 'race_date', 'lane', 'country', 'name', 
                '500m', '1000m', '1500m', '2000m', 'rank', 'code']]

# Save new file for further analysis
df_aux.to_csv(dirname + 'df_resultsfiles.csv', encoding='UTF-8', index=False)

## 1.3. Media start list files

### General  information:  

Here, I have scraped the data from the PDFs, to form a single data frame to be used in the analysis.

### Extract all date of birth data from the PDF and save to a new data frame.

In [ ]:
%time
#Funções auxiliares
# Determine heading
def heading(full):
    venue = full[0].split()[0]
    boat_class = full[1].split()[-1]
    final = full[2].split()[-1]
    race_date = full[3][-20:-9]

    # Establish format behaviour
    boat =[]
    for i in boat_class:
        if i == '8':
            boat=8
        elif i == '4':
            boat=4
        elif i == '2':
            boat=2

    return boat_class, venue, boat, final, race_date

#Extract info based on race position or lane
# Doubles
def first_place_double(races):
        try:
                lane = races[0].split()[0]
                country = races[0].split()[1]
                seat_b = races[0].split()[2]
                seat_s = races[1].split()[0]

                surname_b = races[0].split()[3]
                surname_s = races[1].split()[1]

                name_b = races[0].split()[4]
                name_s = races[1].split()[2]

                #GET date of births
                b_day=races[0].split()[-3]
                b_month=races[0].split()[-2]
                b_year=races[0].split()[-1]
                dob_b = b_day + ' ' + b_month + ' ' + b_year
                s_day=races  [1].split()[-3]
                s_month=races[1].split()[-2]
                s_year=races [1].split()[-1]
                dob_s = s_day + ' ' + s_month + ' ' + s_year


                f_name_b = surname_b + ' ' + name_b
                f_name_s = surname_s + ' ' + name_s

                return [lane, country, seat_b,seat_s, 
                        f_name_b,f_name_s, dob_b,
                        dob_s
                        ]

        except:
                pass

def second_place_double(races):

        try:
                lane = races[2].split()[0]
                country = races[2].split()[1]
                seat_b = races[2].split()[2]
                seat_s = races[3].split()[0]

                surname_b = races[2].split()[3]
                surname_s = races[3].split()[1]

                name_b = races[2].split()[4]
                name_s = races[3].split()[2]

                #GET date of births
                b_day=races[2].split()[-3]
                b_month=races[2].split()[-2]
                b_year=races[2].split()[-1]
                dob_b = b_day + ' ' + b_month + ' ' + b_year
                s_day=races[3].split()[-3]
                s_month=races[3].split()[-2]
                s_year=races[3].split()[-1]
                dob_s = s_day + ' ' + s_month + ' ' + s_year


                f_name_b = surname_b + ' ' + name_b
                f_name_s = surname_s + ' ' + name_s

                return [lane, country, seat_b,seat_s, 
                        f_name_b,f_name_s, dob_b,
                        dob_s
                        ]

        except:
                pass

def third_place_double(races):

        try:
                lane = races[4].split()[0]
                country = races[4].split()[1]
                seat_b = races[4].split()[2]
                seat_s = races[5].split()[0]

                surname_b = races[4].split()[3]
                surname_s = races[5].split()[1]

                name_b = races[4].split()[4]
                name_s = races[5].split()[2]

                #GET date of births
                b_day=races[4].split()[-3]
                b_month=races[4].split()[-2]
                b_year=races[4].split()[-1]
                dob_b = b_day + ' ' + b_month + ' ' + b_year
                s_day=races[5].split()[-3]
                s_month=races[5].split()[-2]
                s_year=races[5].split()[-1]
                dob_s = s_day + ' ' + s_month + ' ' + s_year


                f_name_b = surname_b + ' ' + name_b
                f_name_s = surname_s + ' ' + name_s

                return [lane, country, seat_b,seat_s, 
                        f_name_b,f_name_s, dob_b,
                        dob_s
                        ]

        except:
                pass

def fourth_place_double(races):
        
        try:
                lane = races[6].split()[0]
                country = races[6].split()[1]
                seat_b = races[6].split()[2]
                seat_s = races[7].split()[0]

                surname_b = races[6].split()[3]
                surname_s = races[7].split()[1]

                name_b = races[6].split()[4]
                name_s = races[7].split()[2]

                #GET date of births
                b_day=races[6].split()[-3]
                b_month=races[6].split()[-2]
                b_year=races[6].split()[-1]
                dob_b = b_day + ' ' + b_month + ' ' + b_year
                s_day=races[7].split()[-3]
                s_month=races[7].split()[-2]
                s_year=races[7].split()[-1]
                dob_s = s_day + ' ' + s_month + ' ' + s_year


                f_name_b = surname_b + ' ' + name_b
                f_name_s = surname_s + ' ' + name_s

                return [lane, country, seat_b,seat_s, 
                        f_name_b,f_name_s, dob_b,
                        dob_s
                        ]

        except:
                pass

def fifth_place_double(races):

        try:
                lane = races[8].split()[0]
                country = races[8].split()[1]
                seat_b = races[8].split()[2]
                seat_s = races[9].split()[0]

                surname_b = races[8].split()[3]
                surname_s = races[9].split()[1]

                name_b = races[8].split()[4]
                name_s = races[9].split()[2]

                #GET date of births
                b_day=races[8].split()[-3]
                b_month=races[8].split()[-2]
                b_year=races[8].split()[-1]
                dob_b = b_day + ' ' + b_month + ' ' + b_year
                s_day=races[9].split()[-3]
                s_month=races[9].split()[-2]
                s_year=races[9].split()[-1]
                dob_s = s_day + ' ' + s_month + ' ' + s_year


                f_name_b = surname_b + ' ' + name_b
                f_name_s = surname_s + ' ' + name_s

                return [lane, country, seat_b,seat_s, 
                        f_name_b,f_name_s, dob_b,
                        dob_s
                        ]

        except:
                pass

def sixth_place_double(races):

        try:
                lane = races[10].split()[0]
                country = races[10].split()[1]
                seat_b = races[10].split()[2]
                seat_s = races[11].split()[0]

                surname_b = races[10].split()[3]
                surname_s = races[11].split()[1]

                name_b = races[10].split()[4]
                name_s = races[11].split()[2]

                #GET date of births
                b_day=races[10].split()[-3]
                b_month=races[10].split()[-2]
                b_year=races[10].split()[-1]
                dob_b = b_day + ' ' + b_month + ' ' + b_year
                s_day=races[11].split()[-3]
                s_month=races[11].split()[-2]
                s_year=races[11].split()[-1]
                dob_s = s_day + ' ' + s_month + ' ' + s_year


                f_name_b = surname_b + ' ' + name_b
                f_name_s = surname_s + ' ' + name_s

                return [lane, country, seat_b,seat_s, 
                        f_name_b,f_name_s, dob_b,
                        dob_s
                        ]

        except:
                pass


#Fours
def first_place_four(races):
        try:
                lane = races[0].split()[0]
                country = races[0].split()[1]

                seat_b = races[0].split()[2]
                seat_2 = races[1].split()[0]
                seat_3 = races[2].split()[0]
                seat_s = races[3].split()[0]

                surname_b = races[0].split()[3]
                surname_2 = races[1].split()[1]
                surname_3 = races[2].split()[1]
                surname_s = races[3].split()[1]

                name_b = races[0].split()[4]
                name_2 = races[1].split()[2]
                name_3 = races[2].split()[2]
                name_s = races[3].split()[2]
        
                #GET date of births
                b_day=races[0].split()[-3]
                b_month=races[0].split()[-2]
                b_year=races[0].split()[-1]
                dob_b = b_day+' '+ b_month + ' ' + b_year
                _2_day=races[1].split()[-3]
                _2_month=races[1].split()[-2]
                _2_year=races[1].split()[-1]
                dob_2 = _2_day+' '+ _2_month + ' ' + _2_year
                _3_day=races[2].split()[-3]
                _3_month=races[2].split()[-2]
                _3_year=races[2].split()[-1]
                dob_3 = _3_day+' '+ _3_month + ' ' + _3_year
                s_day=races[3].split()[-3]
                s_month=races[3].split()[-2]
                s_year=races[3].split()[-1]
                dob_s = s_day+' '+ s_month + ' ' + s_year
                
                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_s = surname_s + ' ' + name_s
                
                return [lane, country, seat_b,seat_2,seat_3,
                        seat_s,f_name_b,f_name_2,f_name_3,f_name_s,dob_b,
                        dob_2,dob_3,dob_s
                        ]

        except:
                pass

def second_place_four(races):
        try:
                lane = races[4].split()[0]
                country = races[4].split()[1]

                seat_b = races[4].split()[2]
                seat_2 = races[5].split()[0]
                seat_3 = races[6].split()[0]
                seat_s = races[7].split()[0]

                surname_b = races[4].split()[3]
                surname_2 = races[5].split()[1]
                surname_3 = races[6].split()[1]
                surname_s = races[7].split()[1]

                name_b = races[4].split()[4]
                name_2 = races[5].split()[2]
                name_3 = races[6].split()[2]
                name_s = races[7].split()[2]
        
                #GET date of births
                b_day=races[4].split()[-3]
                b_month=races[4].split()[-2]
                b_year=races[4].split()[-1]
                dob_b = b_day + ' ' + b_month + ' ' + b_year
                _2_day=races[5].split()[-3]
                _2_month=races[5].split()[-2]
                _2_year=races[5].split()[-1]
                dob_2 = _2_day + ' ' + _2_month + ' ' + _2_year
                _3_day=races[6].split()[-3]
                _3_month=races[6].split()[-2]
                _3_year=races[6].split()[-1]
                dob_3 = _3_day + ' ' + _3_month + ' ' + _3_year
                s_day=races[7].split()[-3]
                s_month=races[7].split()[-2]
                s_year=races[7].split()[-1]
                dob_s = s_day + ' ' + s_month + ' ' + s_year
                
                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_s = surname_s + ' ' + name_s
                
                return [lane, country, seat_b,seat_2,seat_3,
                        seat_s,f_name_b,f_name_2,f_name_3,f_name_s,dob_b,
                        dob_2,dob_3,dob_s
                        ]

        except:
                pass

def third_place_four(races):
        try:
                lane = races[8].split()[0]
                country = races[8].split()[1]

                seat_b = races[8].split()[2]
                seat_2 = races[9].split()[0]
                seat_3 = races[10].split()[0]
                seat_s = races[11].split()[0]

                surname_b = races[8].split()[3]
                surname_2 = races[9].split()[1]
                surname_3 = races[10].split()[1]
                surname_s = races[11].split()[1]

                name_b = races[8].split()[4]
                name_2 = races[9].split()[2]
                name_3 = races[10].split()[2]
                name_s = races[11].split()[2]
        
                #GET date of births
                b_day=races[8].split()[-3]
                b_month=races[8].split()[-2]
                b_year=races[8].split()[-1]
                dob_b = b_day + ' ' + b_month + ' ' + b_year
                _2_day=races[9].split()[-3]
                _2_month=races[9].split()[-2]
                _2_year=races[9].split()[-1]
                dob_2 = _2_day + ' ' + _2_month + ' ' + _2_year
                _3_day=races[10].split()[-3]
                _3_month=races[10].split()[-2]
                _3_year=races[10].split()[-1]
                dob_3 = _3_day + ' ' + _3_month + ' ' + _3_year
                s_day=races[11].split()[-3]
                s_month=races[11].split()[-2]
                s_year=races[11].split()[-1]
                dob_s = s_day + ' ' + s_month + ' ' + s_year
                
                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_s = surname_s + ' ' + name_s
                
                return [lane, country, seat_b,seat_2,seat_3,
                        seat_s,f_name_b,f_name_2,f_name_3,f_name_s,dob_b,
                        dob_2,dob_3,dob_s
                        ]

        except:
                pass

def fourth_place_four(races):
        try:
                lane = races[12].split()[0]
                country = races[12].split()[1]

                seat_b = races[12].split()[2]
                seat_2 = races[13].split()[0]
                seat_3 = races[14].split()[0]
                seat_s = races[15].split()[0]

                surname_b = races[12].split()[3]
                surname_2 = races[13].split()[1]
                surname_3 = races[14].split()[1]
                surname_s = races[15].split()[1]

                name_b = races[12].split()[4]
                name_2 = races[13].split()[2]
                name_3 = races[14].split()[2]
                name_s = races[15].split()[2]
        
                #GET date of births
                b_day=races[12].split()[-3]
                b_month=races[12].split()[-2]
                b_year=races[12].split()[-1]
                dob_b = b_day + ' ' + b_month + ' ' + b_year
                _2_day=races[13].split()[-3]
                _2_month=races[13].split()[-2]
                _2_year=races[13].split()[-1]
                dob_2 = _2_day + ' ' + _2_month + ' ' + _2_year
                _3_day=races[14].split()[-3]
                _3_month=races[14].split()[-2]
                _3_year=races[14].split()[-1]
                dob_3 = _3_day + ' ' + _3_month + ' ' + _3_year
                s_day=races[15].split()[-3]
                s_month=races[15].split()[-2]
                s_year=races[15].split()[-1]
                dob_s = s_day + ' ' + s_month + ' ' + s_year
                
                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_s = surname_s + ' ' + name_s
                
                return [lane, country, seat_b,seat_2,seat_3,
                        seat_s,f_name_b,f_name_2,f_name_3,f_name_s,dob_b,
                        dob_2,dob_3,dob_s
                        ]

        except:
                pass

def fifth_place_four(races):
        try:
                lane = races[16].split()[0]
                country = races[16].split()[1]

                seat_b = races[16].split()[2]
                seat_2 = races[17].split()[0]
                seat_3 = races[18].split()[0]
                seat_s = races[19].split()[0]

                surname_b = races[16].split()[3]
                surname_2 = races[17].split()[1]
                surname_3 = races[18].split()[1]
                surname_s = races[19].split()[1]

                name_b = races[16].split()[4]
                name_2 = races[17].split()[2]
                name_3 = races[18].split()[2]
                name_s = races[19].split()[2]
        
                #GET date of births
                b_day=races[16].split()[-3]
                b_month=races[16].split()[-2]
                b_year=races[16].split()[-1]
                dob_b = b_day + ' ' + b_month + ' ' + b_year
                _2_day=races[17].split()[-3]
                _2_month=races[17].split()[-2]
                _2_year=races[17].split()[-1]
                dob_2 = _2_day + ' ' + _2_month + ' ' + _2_year
                _3_day=races[18].split()[-3]
                _3_month=races[18].split()[-2]
                _3_year=races[18].split()[-1]
                dob_3 = _3_day + ' ' + _3_month + ' ' + _3_year
                s_day=races[19].split()[-3]
                s_month=races[19].split()[-2]
                s_year=races[19].split()[-1]
                dob_s = s_day + ' ' + s_month + ' ' + s_year
                
                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_s = surname_s + ' ' + name_s
                
                return [lane, country, seat_b,seat_2,seat_3,
                        seat_s,f_name_b,f_name_2,f_name_3,f_name_s,dob_b,
                        dob_2,dob_3,dob_s
                        ]

        except:
                pass

def sixth_place_four(races):
        try:
                lane = races[20].split()[0]
                country = races[20].split()[1]

                seat_b = races[20].split()[2]
                seat_2 = races[21].split()[0]
                seat_3 = races[22].split()[0]
                seat_s = races[23].split()[0]

                surname_b = races[20].split()[3]
                surname_2 = races[21].split()[1]
                surname_3 = races[22].split()[1]
                surname_s = races[23].split()[1]

                name_b = races[20].split()[4]
                name_2 = races[21].split()[2]
                name_3 = races[22].split()[2]
                name_s = races[23].split()[2]
        
                #GET date of births
                b_day=races[20].split()[-3]
                b_month=races[20].split()[-2]
                b_year=races[20].split()[-1]
                dob_b = b_day + ' ' + b_month + ' ' + b_year
                _2_day=races[21].split()[-3]
                _2_month=races[21].split()[-2]
                _2_year=races[21].split()[-1]
                dob_2 = _2_day + ' ' + _2_month + ' ' + _2_year
                _3_day=races[22].split()[-3]
                _3_month=races[22].split()[-2]
                _3_year=races[22].split()[-1]
                dob_3 = _3_day + ' ' + _3_month + ' ' + _3_year
                s_day=races[23].split()[-3]
                s_month=races[23].split()[-2]
                s_year=races[23].split()[-1]
                dob_s = s_day + ' ' + s_month + ' ' + s_year
                
                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_s = surname_s + ' ' + name_s
                
                return [lane, country, seat_b,seat_2,seat_3,
                        seat_s,f_name_b,f_name_2,f_name_3,f_name_s,dob_b,
                        dob_2,dob_3,dob_s
                        ]

        except:
                pass


#Eights
def first_place_eight(races):
        try:
                lane = races[0].split()[0]
                country = races[0].split()[1]

                seat_b = races[0].split()[2]
                seat_2 = races[1].split()[0]
                seat_3 = races[2].split()[0]
                seat_4 = races[3].split()[0]
                seat_5 = races[4].split()[0]
                seat_6 = races[5].split()[0]
                seat_7 = races[6].split()[0]
                seat_s = races[7].split()[0]
                seat_c = races[8].split()[0]

                surname_b = races[0].split()[3]
                surname_2 = races[1].split()[1]
                surname_3 = races[2].split()[1]
                surname_4 = races[3].split()[1]
                surname_5 = races[4].split()[1]
                surname_6 = races[5].split()[1]
                surname_7 = races[6].split()[1]
                surname_s = races[7].split()[1]
                surname_c = races[8].split()[1]

                name_b = races[0].split()[4]
                name_2 = races[1].split()[2]
                name_3 = races[2].split()[2]
                name_4 = races[3].split()[2]
                name_5 = races[4].split()[2]
                name_6 = races[5].split()[2]
                name_7 = races[6].split()[2]
                name_s = races[7].split()[2]
                name_c = races[8].split()[2]

                #GET date of births
                b_day=races[0].split()[-3]
                b_month=races[0].split()[-2]
                b_year=races[0].split()[-1]
                dob_b = b_day+' '+ b_month + ' ' + b_year
                _2_day=races[1].split()[-3]
                _2_month=races[1].split()[-2]
                _2_year=races[1].split()[-1]
                dob_2 = _2_day+' '+ _2_month + ' ' + _2_year
                _3_day=races[2].split()[-3]
                _3_month=races[2].split()[-2]
                _3_year=races[2].split()[-1]
                dob_3 = _3_day+' '+ _3_month + ' ' + _3_year
                _4_day=races[3].split()[-3]
                _4_month=races[3].split()[-2]
                _4_year=races[3].split()[-1]
                dob_4 = _4_day+' '+ _4_month + ' ' + _4_year
                _5_day=races[4].split()[-3]
                _5_month=races[4].split()[-2]
                _5_year=races[4].split()[-1]
                dob_5 = _5_day+' '+ _5_month + ' ' + _5_year
                _6_day=races[5].split()[-3]
                _6_month=races[5].split()[-2]
                _6_year=races[5].split()[-1]
                dob_6 = _6_day+' '+ _6_month + ' ' + _6_year
                _7_day=races[6].split()[-3]
                _7_month=races[6].split()[-2]
                _7_year=races[6].split()[-1]
                dob_7 = _7_day+' '+ _7_month + ' ' + _7_year
                s_day=races[7].split()[-3]
                s_month=races[7].split()[-2]
                s_year=races[7].split()[-1]
                dob_s = s_day+' '+ s_month + ' ' + s_year
                c_day=races[8].split()[-3]
                c_month=races[8].split()[-2]
                c_year=races[8].split()[-1]
                dob_c = c_day+' '+ c_month + ' ' + c_year

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_4 = surname_4 + ' ' + name_4
                f_name_5 = surname_5 + ' ' + name_5
                f_name_6 = surname_6 + ' ' + name_6
                f_name_7 = surname_7 + ' ' + name_7
                f_name_s = surname_s + ' ' + name_s
                f_name_c = surname_c + ' ' + name_c

                return [lane, country, seat_b,seat_2,seat_3,
                        seat_4,seat_5,seat_6,seat_7,seat_s, seat_c, 
                        f_name_b,f_name_2,f_name_3,f_name_4,f_name_5,
                        f_name_6,f_name_7,f_name_s,f_name_c, dob_b,
                        dob_2,dob_3,dob_4,dob_5,dob_6,dob_7,dob_s,dob_c
                        ]

        except:
                pass

def second_place_eight(races):
        try:
                lane = races[9].split()[0]
                country = races[9].split()[1]

                seat_b = races[9].split()[2]
                seat_2 = races[10].split()[0]
                seat_3 = races[11].split()[0]
                seat_4 = races[12].split()[0]
                seat_5 = races[13].split()[0]
                seat_6 = races[14].split()[0]
                seat_7 = races[15].split()[0]
                seat_s = races[16].split()[0]
                seat_c = races[17].split()[0]

                surname_b = races[9].split()[3]
                surname_2 = races[10].split()[1]
                surname_3 = races[11].split()[1]
                surname_4 = races[12].split()[1]
                surname_5 = races[13].split()[1]
                surname_6 = races[14].split()[1]
                surname_7 = races[15].split()[1]
                surname_s = races[16].split()[1]
                surname_c = races[17].split()[1]

                name_b = races[9].split()[4]
                name_2 = races[10].split()[2]
                name_3 = races[11].split()[2]
                name_4 = races[12].split()[2]
                name_5 = races[13].split()[2]
                name_6 = races[14].split()[2]
                name_7 = races[15].split()[2]
                name_s = races[16].split()[2]
                name_c = races[17].split()[2]

                #GET date of births
                b_day=races[9].split()[-3]
                b_month=races[9].split()[-2]
                b_year=races[9].split()[-1]
                dob_b = b_day+' '+ b_month + ' ' + b_year
                _2_day=races[10].split()[-3]
                _2_month=races[10].split()[-2]
                _2_year=races[10].split()[-1]
                dob_2 = _2_day+' '+ _2_month + ' ' + _2_year
                _3_day=races[11].split()[-3]
                _3_month=races[11].split()[-2]
                _3_year=races[11].split()[-1]
                dob_3 = _3_day+' '+ _3_month + ' ' + _3_year
                _4_day=races[12].split()[-3]
                _4_month=races[12].split()[-2]
                _4_year=races[12].split()[-1]
                dob_4 = _4_day+' '+ _4_month + ' ' + _4_year
                _5_day=races[13].split()[-3]
                _5_month=races[13].split()[-2]
                _5_year=races[13].split()[-1]
                dob_5 = _5_day+' '+ _5_month + ' ' + _5_year
                _6_day=races[14].split()[-3]
                _6_month=races[14].split()[-2]
                _6_year=races[14].split()[-1]
                dob_6 = _6_day+' '+ _6_month + ' ' + _6_year
                _7_day=races[15].split()[-3]
                _7_month=races[15].split()[-2]
                _7_year=races[15].split()[-1]
                dob_7 = _7_day+' '+ _7_month + ' ' + _7_year
                s_day=races[16].split()[-3]
                s_month=races[16].split()[-2]
                s_year=races[16].split()[-1]
                dob_s = s_day+' '+ s_month + ' ' + s_year
                c_day=races[17].split()[-3]
                c_month=races[17].split()[-2]
                c_year=races[17].split()[-1]
                dob_c = c_day+' '+ c_month + ' ' + c_year

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_4 = surname_4 + ' ' + name_4
                f_name_5 = surname_5 + ' ' + name_5
                f_name_6 = surname_6 + ' ' + name_6
                f_name_7 = surname_7 + ' ' + name_7
                f_name_s = surname_s + ' ' + name_s
                f_name_c = surname_c + ' ' + name_c

                return [lane, country, seat_b,seat_2,seat_3,
                        seat_4,seat_5,seat_6,seat_7,seat_s, seat_c, 
                        f_name_b,f_name_2,f_name_3,f_name_4,f_name_5,
                        f_name_6,f_name_7,f_name_s,f_name_c, dob_b,
                        dob_2,dob_3,dob_4,dob_5,dob_6,dob_7,dob_s,dob_c
                        ]

        except:
                pass

def third_place_eight(races):
        try:
                lane = races[18].split()[0]
                country = races[18].split()[1]

                seat_b = races[18].split()[2]
                seat_2 = races[19].split()[0]
                seat_3 = races[20].split()[0]
                seat_4 = races[21].split()[0]
                seat_5 = races[22].split()[0]
                seat_6 = races[23].split()[0]
                seat_7 = races[24].split()[0]
                seat_s = races[25].split()[0]
                seat_c = races[26].split()[0]

                surname_b = races[18].split()[3]
                surname_2 = races[19].split()[1]
                surname_3 = races[20].split()[1]
                surname_4 = races[21].split()[1]
                surname_5 = races[22].split()[1]
                surname_6 = races[23].split()[1]
                surname_7 = races[24].split()[1]
                surname_s = races[25].split()[1]
                surname_c = races[26].split()[1]

                name_b = races[18].split()[4]
                name_2 = races[19].split()[2]
                name_3 = races[20].split()[2]
                name_4 = races[21].split()[2]
                name_5 = races[22].split()[2]
                name_6 = races[23].split()[2]
                name_7 = races[24].split()[2]
                name_s = races[25].split()[2]
                name_c = races[26].split()[2]

                #GET date of births
                b_day=races[18].split()[-3]
                b_month=races[18].split()[-2]
                b_year=races[18].split()[-1]
                dob_b = b_day+' '+ b_month + ' ' + b_year
                _2_day=races[19].split()[-3]
                _2_month=races[19].split()[-2]
                _2_year=races[19].split()[-1]
                dob_2 = _2_day+' '+ _2_month + ' ' + _2_year
                _3_day=races[20].split()[-3]
                _3_month=races[20].split()[-2]
                _3_year=races[20].split()[-1]
                dob_3 = _3_day+' '+ _3_month + ' ' + _3_year
                _4_day=races[21].split()[-3]
                _4_month=races[21].split()[-2]
                _4_year=races[21].split()[-1]
                dob_4 = _4_day+' '+ _4_month + ' ' + _4_year
                _5_day=races[22].split()[-3]
                _5_month=races[22].split()[-2]
                _5_year=races[22].split()[-1]
                dob_5 = _5_day+' '+ _5_month + ' ' + _5_year
                _6_day=races[23].split()[-3]
                _6_month=races[23].split()[-2]
                _6_year=races[23].split()[-1]
                dob_6 = _6_day+' '+ _6_month + ' ' + _6_year
                _7_day=races[24].split()[-3]
                _7_month=races[24].split()[-2]
                _7_year=races[24].split()[-1]
                dob_7 = _7_day+' '+ _7_month + ' ' + _7_year
                s_day=races[25].split()[-3]
                s_month=races[25].split()[-2]
                s_year=races[25].split()[-1]
                dob_s = s_day+' '+ s_month + ' ' + s_year
                c_day=races[26].split()[-3]
                c_month=races[26].split()[-2]
                c_year=races[26].split()[-1]
                dob_c = c_day+' '+ c_month + ' ' + c_year

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_4 = surname_4 + ' ' + name_4
                f_name_5 = surname_5 + ' ' + name_5
                f_name_6 = surname_6 + ' ' + name_6
                f_name_7 = surname_7 + ' ' + name_7
                f_name_s = surname_s + ' ' + name_s
                f_name_c = surname_c + ' ' + name_c

                return [lane, country, seat_b,seat_2,seat_3,
                        seat_4,seat_5,seat_6,seat_7,seat_s, seat_c, 
                        f_name_b,f_name_2,f_name_3,f_name_4,f_name_5,
                        f_name_6,f_name_7,f_name_s,f_name_c, dob_b,
                        dob_2,dob_3,dob_4,dob_5,dob_6,dob_7,dob_s,dob_c
                        ]

        except:
                pass

def fourth_place_eight(races):
        try:
                lane = races[27].split()[0]
                country = races[27].split()[1]

                seat_b = races[27].split()[2]
                seat_2 = races[28].split()[0]
                seat_3 = races[29].split()[0]
                seat_4 = races[30].split()[0]
                seat_5 = races[31].split()[0]
                seat_6 = races[32].split()[0]
                seat_7 = races[33].split()[0]
                seat_s = races[34].split()[0]
                seat_c = races[35].split()[0]

                surname_b = races[27].split()[3]
                surname_2 = races[28].split()[1]
                surname_3 = races[29].split()[1]
                surname_4 = races[30].split()[1]
                surname_5 = races[31].split()[1]
                surname_6 = races[32].split()[1]
                surname_7 = races[33].split()[1]
                surname_s = races[34].split()[1]
                surname_c = races[35].split()[1]

                name_b = races[27].split()[4]
                name_2 = races[28].split()[2]
                name_3 = races[29].split()[2]
                name_4 = races[30].split()[2]
                name_5 = races[31].split()[2]
                name_6 = races[32].split()[2]
                name_7 = races[33].split()[2]
                name_s = races[34].split()[2]
                name_c = races[35].split()[2]
                # print(name)

                #GET date of births
                b_day=races[27].split()[-3]
                b_month=races[27].split()[-2]
                b_year=races[27].split()[-1]
                dob_b = b_day+' '+ b_month + ' ' + b_year
                _2_day=races[28].split()[-3]
                _2_month=races[28].split()[-2]
                _2_year=races[28].split()[-1]
                dob_2 = _2_day+' '+ _2_month + ' ' + _2_year
                _3_day=races[29].split()[-3]
                _3_month=races[29].split()[-2]
                _3_year=races[29].split()[-1]
                dob_3 = _3_day+' '+ _3_month + ' ' + _3_year
                _4_day=races[30].split()[-3]
                _4_month=races[30].split()[-2]
                _4_year=races[30].split()[-1]
                dob_4 = _4_day+' '+ _4_month + ' ' + _4_year
                _5_day=races[31].split()[-3]
                _5_month=races[31].split()[-2]
                _5_year=races[31].split()[-1]
                dob_5 = _5_day+' '+ _5_month + ' ' + _5_year
                _6_day=races[32].split()[-3]
                _6_month=races[32].split()[-2]
                _6_year=races[32].split()[-1]
                dob_6 = _6_day+' '+ _6_month + ' ' + _6_year
                _7_day=races[33].split()[-3]
                _7_month=races[33].split()[-2]
                _7_year=races[33].split()[-1]
                dob_7 = _7_day+' '+ _7_month + ' ' + _7_year
                s_day=races[34].split()[-3]
                s_month=races[34].split()[-2]
                s_year=races[34].split()[-1]
                dob_s = s_day+' '+ s_month + ' ' + s_year
                c_day=races[35].split()[-3]
                c_month=races[35].split()[-2]
                c_year=races[35].split()[-1]
                dob_c = c_day+' '+ c_month + ' ' + c_year

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_4 = surname_4 + ' ' + name_4
                f_name_5 = surname_5 + ' ' + name_5
                f_name_6 = surname_6 + ' ' + name_6
                f_name_7 = surname_7 + ' ' + name_7
                f_name_s = surname_s + ' ' + name_s
                f_name_c = surname_c + ' ' + name_c

                return [lane, country, seat_b,seat_2,seat_3,
                        seat_4,seat_5,seat_6,seat_7,seat_s, seat_c, 
                        f_name_b,f_name_2,f_name_3,f_name_4,f_name_5,
                        f_name_6,f_name_7,f_name_s,f_name_c, dob_b,
                        dob_2,dob_3,dob_4,dob_5,dob_6,dob_7,dob_s,dob_c
                        ]

        except:
                pass

def fifth_place_eight(races):
        try:
                lane = races[36].split()[0]
                country = races[36].split()[1]

                seat_b = races[36].split()[2]
                seat_2 = races[37].split()[0]
                seat_3 = races[38].split()[0]
                seat_4 = races[39].split()[0]
                seat_5 = races[40].split()[0]
                seat_6 = races[41].split()[0]
                seat_7 = races[42].split()[0]
                seat_s = races[43].split()[0]
                seat_c = races[44].split()[0]

                surname_b = races[36].split()[3]
                surname_2 = races[37].split()[1]
                surname_3 = races[38].split()[1]
                surname_4 = races[39].split()[1]
                surname_5 = races[40].split()[1]
                surname_6 = races[41].split()[1]
                surname_7 = races[42].split()[1]
                surname_s = races[43].split()[1]
                surname_c = races[44].split()[1]

                name_b = races[36].split()[4]
                name_2 = races[37].split()[2]
                name_3 = races[38].split()[2]
                name_4 = races[39].split()[2]
                name_5 = races[40].split()[2]
                name_6 = races[41].split()[2]
                name_7 = races[42].split()[2]
                name_s = races[43].split()[2]
                name_c = races[44].split()[2]

                #GET date of births
                b_day=races  [36].split()[-3]
                b_month=races[36].split()[-2]
                b_year=races [36].split()[-1]
                dob_b = b_day+' '+ b_month + ' ' + b_year
                _2_day=races  [37].split()[-3]
                _2_month=races[37].split()[-2]
                _2_year=races [37].split()[-1]
                dob_2 = _2_day+' '+ _2_month + ' ' + _2_year
                _3_day=races  [38].split()[-3]
                _3_month=races[38].split()[-2]
                _3_year=races [38].split()[-1]
                dob_3 = _3_day+' '+ _3_month + ' ' + _3_year
                _4_day=races  [39].split()[-3]
                _4_month=races[39].split()[-2]
                _4_year=races [39].split()[-1]
                dob_4 = _4_day+' '+ _4_month + ' ' + _4_year
                _5_day=races  [40].split()[-3]
                _5_month=races[40].split()[-2]
                _5_year=races [40].split()[-1]
                dob_5 = _5_day+' '+ _5_month + ' ' + _5_year
                _6_day=races  [41].split()[-3]
                _6_month=races[41].split()[-2]
                _6_year=races [41].split()[-1]
                dob_6 = _6_day+' '+ _6_month + ' ' + _6_year
                _7_day=races  [42].split()[-3]
                _7_month=races[42].split()[-2]
                _7_year=races [42].split()[-1]
                dob_7 = _7_day+' '+ _7_month + ' ' + _7_year
                s_day=races  [43].split()[-3]
                s_month=races[43].split()[-2]
                s_year=races [43].split()[-1]
                dob_s = s_day+' '+ s_month + ' ' + s_year
                c_day=races  [44].split()[-3]
                c_month=races[44].split()[-2]
                c_year=races [44].split()[-1]
                dob_c = c_day+' '+ c_month + ' ' + c_year

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_4 = surname_4 + ' ' + name_4
                f_name_5 = surname_5 + ' ' + name_5
                f_name_6 = surname_6 + ' ' + name_6
                f_name_7 = surname_7 + ' ' + name_7
                f_name_s = surname_s + ' ' + name_s
                f_name_c = surname_c + ' ' + name_c

                return [lane, country, seat_b,seat_2,seat_3,
                        seat_4,seat_5,seat_6,seat_7,seat_s, seat_c, 
                        f_name_b,f_name_2,f_name_3,f_name_4,f_name_5,
                        f_name_6,f_name_7,f_name_s,f_name_c, dob_b,
                        dob_2,dob_3,dob_4,dob_5,dob_6,dob_7,dob_s,dob_c
                        ]

        except:
                pass

def sixth_place_eight(races):
        try:
                lane = races[45].split()[0]
                country = races[45].split()[1]

                seat_b = races[45].split()[2]
                seat_2 = races[46].split()[0]
                seat_3 = races[47].split()[0]
                seat_4 = races[48].split()[0]
                seat_5 = races[49].split()[0]
                seat_6 = races[50].split()[0]
                seat_7 = races[51].split()[0]
                seat_s = races[52].split()[0]
                seat_c = races[53].split()[0]

                surname_b = races[45].split()[3]
                surname_2 = races[46].split()[1]
                surname_3 = races[47].split()[1]
                surname_4 = races[48].split()[1]
                surname_5 = races[49].split()[1]
                surname_6 = races[50].split()[1]
                surname_7 = races[51].split()[1]
                surname_s = races[52].split()[1]
                surname_c = races[53].split()[1]

                name_b = races[45].split()[4]
                name_2 = races[46].split()[2]
                name_3 = races[47].split()[2]
                name_4 = races[48].split()[2]
                name_5 = races[49].split()[2]
                name_6 = races[50].split()[2]
                name_7 = races[51].split()[2]
                name_s = races[52].split()[2]
                name_c = races[53].split()[2]

                #GET date of births
                b_day=races  [45].split()[-3]
                b_month=races[45].split()[-2]
                b_year=races [45].split()[-1]
                dob_b = b_day+' '+ b_month + ' ' + b_year
                _2_day=races  [46].split()[-3]
                _2_month=races[46].split()[-2]
                _2_year=races [46].split()[-1]
                dob_2 = _2_day+' '+ _2_month + ' ' + _2_year
                _3_day=races  [47].split()[-3]
                _3_month=races[47].split()[-2]
                _3_year=races [47].split()[-1]
                dob_3 = _3_day+' '+ _3_month + ' ' + _3_year
                _4_day=races  [48].split()[-3]
                _4_month=races[48].split()[-2]
                _4_year=races [48].split()[-1]
                dob_4 = _4_day+' '+ _4_month + ' ' + _4_year
                _5_day=races  [49].split()[-3]
                _5_month=races[49].split()[-2]
                _5_year=races [49].split()[-1]
                dob_5 = _5_day+' '+ _5_month + ' ' + _5_year
                _6_day=races  [50].split()[-3]
                _6_month=races[50].split()[-2]
                _6_year=races [50].split()[-1]
                dob_6 = _6_day+' '+ _6_month + ' ' + _6_year
                _7_day=races  [51].split()[-3]
                _7_month=races[51].split()[-2]
                _7_year=races [51].split()[-1]
                dob_7 = _7_day+' '+ _7_month + ' ' + _7_year
                s_day=races  [52].split()[-3]
                s_month=races[52].split()[-2]
                s_year=races [52].split()[-1]
                dob_s = s_day+' '+ s_month + ' ' + s_year
                c_day=races  [53].split()[-3]
                c_month=races[53].split()[-2]
                c_year=races [53].split()[-1]
                dob_c = c_day+' '+ c_month + ' ' + c_year

                f_name_b = surname_b + ' ' + name_b
                f_name_2 = surname_2 + ' ' + name_2
                f_name_3 = surname_3 + ' ' + name_3
                f_name_4 = surname_4 + ' ' + name_4
                f_name_5 = surname_5 + ' ' + name_5
                f_name_6 = surname_6 + ' ' + name_6
                f_name_7 = surname_7 + ' ' + name_7
                f_name_s = surname_s + ' ' + name_s
                f_name_c = surname_c + ' ' + name_c

                return [lane, country, seat_b,seat_2,seat_3,
                        seat_4,seat_5,seat_6,seat_7,seat_s, seat_c, 
                        f_name_b,f_name_2,f_name_3,f_name_4,f_name_5,
                        f_name_6,f_name_7,f_name_s,f_name_c, dob_b,
                        dob_2,dob_3,dob_4,dob_5,dob_6,dob_7,dob_s,dob_c
                        ]

        except:
                pass


########################################
### Iterate over files starts here #####
########################################

# Establishing directory name where files are:
dirname = '/media/thiago/THIAGO1TB/Thiago/2021/Academico/GABEF/post_doc/Projeto/data/2022/u23/media_start_list/'

# What file extension to look for.
ext = ('.PDF')

# iterating over all files
for pdfname in os.listdir(dirname):
    if pdfname.endswith(ext):
        pdf = dirname + pdfname

        pdf_a = pdfplumber.open(pdf)
        page = pdf_a.pages[0]
        text = page.extract_text()
        full = text.split('\n')

        df_head = heading(full)

        if df_head[2] == 2:
            print(pdf)
            races = full[10:]
            a = first_place_double(races)
            b = second_place_double(races)
            c = third_place_double(races)
            d = fourth_place_double(races)
            e = fifth_place_double(races)
            f = sixth_place_double(races)

            hd = pd.Series(df_head)
            aa = pd.Series(a)
            bb = pd.Series(b)
            cc = pd.Series(c)
            dd = pd.Series(d)
            ee = pd.Series(e)
            ff = pd.Series(f)

            k=pd.concat([aa, bb, cc, dd, ee, ff], axis=1)
            l=pd.concat([hd,k], axis=1).T

            csv_name = df_head[0] + '_' + df_head[3] + '_' + df_head[4][-4:] + '.csv'
            csv_path = dirname + csv_name

            l.to_csv(csv_path, encoding='utf-8', index=False)
            
            

        elif df_head[2] == 4:
            print(pdf)
            races = full[10:]
            a = first_place_four(races)
            b = second_place_four(races)
            c = third_place_four(races)
            d = fourth_place_four(races)
            e = fifth_place_four(races)
            f = sixth_place_four(races)

            hd = pd.Series(df_head)
            aa = pd.Series(a)
            bb = pd.Series(b)
            cc = pd.Series(c)
            dd = pd.Series(d)
            ee = pd.Series(e)
            ff = pd.Series(f)

            k = pd.concat([aa, bb, cc, dd, ee, ff], axis=1)
            l = pd.concat([hd,k], axis=1).T
            
            csv_name = df_head[0] + '_' + df_head[3] + '_' + df_head[4][-4:] + '.csv'
            csv_path = dirname + csv_name

            l.to_csv(csv_path, encoding='utf-8', index=False)

        elif df_head[2] == 8:
            print(pdf)
            races = full[10:]
            a = first_place(races)
            b = second_place(races)
            c = third_place(races)
            d = fourth_place(races)
            e = fifth_place(races)
            f = sixth_place(races)

            hd = pd.Series(df_head)
            aa = pd.Series(a)
            bb = pd.Series(b)
            cc = pd.Series(c)
            dd = pd.Series(d)
            ee = pd.Series(e)
            ff = pd.Series(f)

            k = pd.concat([aa, bb, cc, dd, ee, ff], axis=1)
            l = pd.concat([hd,k], axis=1).T
            
            csv_name = df_head[0] + '_' + df_head[3] + '_' + df_head[4][-4:] + '.csv'
            csv_path = dirname + csv_name

            l.to_csv(csv_path, encoding='utf-8', index=False)

### Processing the .csv file

In [ ]:
# Defining a directory name where the files are stored
dirname = '/media/thiago/THIAGO1TB/Thiago/2021/Academico/GABEF/post_doc/Projeto/data/2022/1x/u23/media_start_list/'

# What file extension to look for.
ext = ('.csv')

list_aux = [] # This is an auxiliary list to gather all the new data frames.

# iterating over all files
for csvname in os.listdir(dirname):
    if csvname.endswith(ext):
        csv_file = dirname + csvname

        df=pd.read_csv(csv_file)

        # The files have information of single, double, four, and eight boat_class. The if/else conditional
        #handles that.
        if df.iloc[0,4] == '1':
            # Creating columns using headings information
            df['age_group'] = df.iloc[0,0]
            df['place'] = df.iloc[0,1]
            df['final'] = df.iloc[0,2]
            df['race_date'] = df.iloc[0,3]

            # Rename the dataframe
            df = df.rename(columns={df.columns[0]: 'lane', df.columns[1]: 'country', df.columns[2]: 'name',
                                    df.columns[3]: 'dob',df.columns[4]: '1'})
        
        elif df.iloc[0,2] == '2':
            # Creating columns using headings information
            df['age_group'] = df.iloc[0,0]
            df['place'] = df.iloc[0,1]
            df['final'] = df.iloc[0,3]
            df['race_date'] = df.iloc[0,4]
            df=df.drop(0)

            # Create a new column, concatenating boat positioning and names
            df['ath_pos_dob'] = df.apply(lambda x: [str(x[i+2]) + '~' + str(x[i+4]) + '~' + str(x[i+6]) for i in range (2)] ,axis=1)

            # From the newly created column, separate the athletes and place their names in separate rows
            #(creating new rows)
            df = df.explode('ath_pos_dob')

            # Separate crewpos from name in two columns
            df[['crewpos', 'name', 'dob']] = df['ath_pos_dob'].str.split('~', expand=True)

            # Drop old, unused columns
            df = df.drop(df.columns[[2,3,4,5,6,7]], axis=1)
            df = df.drop(['ath_pos_dob'], axis=1)
            # 
            # Rename the dataframe
            df = df.rename(columns={df.columns[0]: 'lane', df.columns[1]: 'country'})

        elif df.iloc[0,2] == '4':
            # Creating columns using headings information
            df['age_group'] = df.iloc[0,0]
            df['place'] = df.iloc[0,1]
            df['final'] = df.iloc[0,3]
            df['race_date'] = df.iloc[0,4]
            df=df.drop(0)

            # Create a new column, concatenating boat positioning and names
            df['ath_pos_dob'] = df.apply(lambda x: [str(x[i+2]) + '~' + str(x[i+6]) + '~' + str(x[i+10]) for i in range (4)] ,axis=1)

            # From the newly created column, separate the athletes and place their names in separate rows
            #(creating new rows)
            df = df.explode('ath_pos_dob')

            # Separate crewpos from name in two columns
            df[['crewpos', 'name', 'dob']] = df['ath_pos_dob'].str.split('~', expand=True)

            # Drop old, unused columns
            df = df.drop(df.iloc[:,2:14], axis=1)
            df = df.drop(['ath_pos_dob'], axis=1)
            # 
            # Rename the dataframe
            df = df.rename(columns={df.columns[0]: 'lane', df.columns[1]: 'country'})

                    
        elif df.iloc[0,2] == '8':
            # Creating columns using headings information
            df['age_group'] = df.iloc[0,0]
            df['place'] = df.iloc[0,1]
            df['final'] = df.iloc[0,3]
            df['race_date'] = df.iloc[0,4]
            df=df.drop(0)
            
            # Create a new column, concatenating boat positioning and names
            df['ath_pos_dob'] = df.apply(lambda x: [str(x[i+2]) + '~' + str(x[i+11]) + '~' + str(x[i+20]) for i in range (9)] ,axis=1)
            
            # From the newly created column, separate the athletes and place their names in separate rows
            # (creating new rows)
            df = df.explode('ath_pos_dob')
            
            # Separate crewpos from name in two columns
            df[['crewpos', 'name', 'dob']] = df['ath_pos_dob'].str.split('~', expand=True)
            
            # Drop old, unused columns
            df = df.drop(df.iloc[:, 2:29], axis=1)
            df = df.drop(['ath_pos_dob'], axis=1)
            # 
            # Rename the dataframe
            df = df.rename(columns={df.columns[0]: 'lane', df.columns[1]: 'country'})
    

        else:
            print('Code broke: Please, search for the issue.')

        list_aux.append(df)

df_aux = pd.concat(list_aux)

#Create a varible "code" to be used to merge files
df_aux['code'] = df_aux.apply(lambda x: [str(x[i]) + str(x[i+1]) + str(x[i+2]) + str(x[i+3]) + str(x[i+4]) +
                                str(x[i+5])  for i in range (1)] ,axis=1)

df_aux.to_csv(dirname + 'df_results_mediastartlist.csv', encoding='UTF-8', index=False)